## 라이브러리 import

In [3]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import *
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pickle
from eunjeon import Mecab

In [4]:
# dataframe print 이쁘게
pd.set_option('display.unicode.east_asian_width', True)

## 모델, 인코더, 토크나이저, 데이터 로드

In [5]:
# 모델 로드
model = load_model('/content/drive/MyDrive/python/dl/prj02/models/books_classification_0.9008.h5')

OSError: SavedModel file does not exist at: /content/drive/MyDrive/python/dl/prj02/models/books_classification_0.9008.h5/{saved_model.pbtxt|saved_model.pb}

In [76]:
# Tokenizer 로드
with open('/content/drive/MyDrive/python/dl/prj02/datasets/nouns_token.pickle', 'rb') as f:
    token = pickle.load(f)

In [77]:
# LabelEncoder 로드
with open('/content/drive/MyDrive/python/dl/prj02/datasets/nouns_category_onehot_encoder.pickle', 'rb') as f:
    encoder = pickle.load(f)
category = encoder.categories_[0]
print(category)

['Art_Culture' 'Economic_Management' 'Health_Hobby_Leisure'
 'Home_Cook_Beauty' 'Novel_Poem' 'Reference_Book' 'Religion_Mysticism'
 'Science']


/usr/local/lib/python3.7/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator OneHotEncoder from version 0.24.2 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [78]:
# 데이터 로드
df = pd.read_csv('/content/drive/MyDrive/python/dl/prj02/datasets/raw_test_2021-06-22.csv', index_col=0)
print(df.head())
print(df.info())

                                                                                                                                          summary    category
title                                                                                                                                                        
모험가가 되고 싶다며 도시로 떠났던 딸이 S랭크가 되었다 1...  고향 시골에서 은퇴 모험가 생활을 보내던 벨그리프는 숲에서 주운 소녀를 안젤린이라 ...  Novel_Poem
사이코로지컬 -상                                             니시오 이신의 <헛소리 시리즈>의 네 번째 이야기. 엔터테인먼트 소설 장르로 독자들...    Novel_Poem
혐오자살                                                     제12회 세계문학상을 수상한 조영주 작가가 데뷔 10주년을 맞아 출간한 새로운 미스...     Novel_Poem
미중전쟁 (30만부 돌파 기념 특별 합본판)                      거침없는 문제 제기로 우리 사회의 핫이슈를 정조준해온 작가 김진명의 장편소설 <미중...  Novel_Poem
히페리온의 몰락                                              장르 문학계의 주요 상을 두루 수상하며 평단과 독자의 열광적 지지를 받고 있는 댄 ...    Novel_Poem
<class 'pandas.core.frame.DataFrame'>
Index: 434 entries, 모험가가 되고 싶다며 도시로 떠났던 딸이 S랭크가 되었다 1 to 우리꽃 답사기
Data columns (t

## 전처리

### 중복값 제거

In [79]:
# 결측값 제거
df.reset_index(inplace=True)
df.drop_duplicates(subset=['title'], inplace=True)
df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)
print(df['title'].duplicated().sum())

0


### feature, target 분리

In [80]:
X = df['title']
Y = df[['category']]

### 형태소 분석

In [81]:
# 데이터 형태소 분석
m = Okt()
for i in range(len(X)):
    X[i] = m.nouns(X[i])
print(X)

0      [모험가, 도시, 딸, 랭크, 고향, 시골, 은퇴, 모험가, 생활, 리프, 숲, 소... 
1      [코로, 컬, 상, 시오, 이신, 헛소리, 시리즈, 의, 네, 이야기, 엔터테인먼트...
2      [혐오, 자살, 제, 세계문학, 수상한, 조영주, 작가, 데뷔, 주년, 간, 미스터...
3      [중, 전쟁, 돌파, 기념, 특별, 합본, 판, 문제, 제기로, 우리, 사회, 핫이... 
4      [히페리온, 몰락, 장르, 문학계, 주요, 상, 두루, 평단, 독자, 열광, 지지,...
                                         ...                                    
427    [지구, 재앙, 보고서, 지구온난화, 현실, 전달, 알래스카, 주, 데드, 호스, ...
428    [기회, 확률, 법칙, 주식, 투자, 여론조사, 복권, 당첨, 배우자, 선택, 곳곳...
429    [공기, 정화, 벌레, 잡, 식물, 집안, 공기, 환경, 개선, 기능, 식물, 종,...  
430    [프로, 테오, 믹스, 이제, 생명, 현상, 관, 학문, 분야, 자리, 프로, 테오... 
431    [우리, 꽃, 답, 사기, 우리, 꽃, 사랑, 김태정, 여, 년, 답사, 생활, 소...   
Length: 432, dtype: object


In [82]:
# stopword 로드
stopwords = pd.read_csv('/content/drive/MyDrive/python/dl/prj02/datasets/stopwords.csv', index_col=0)
print(stopwords)

    stopword
0         아
1         휴
2     아이구
3     아이쿠
4     아이고
..       ...
783   없다는
784   않을까
785     있지
786   있으며
787   그대로

[787 rows x 1 columns]


In [83]:
# 함수로 만들어 기존 데이터에 apply해 stopword 제거
def delete_stopwords(lst):
    words = []
    for word in lst:
        if word not in list(stopwords['stopword']) and len(word) > 1:
            words.append(word)
    return ' '.join(words)

X = X.apply(delete_stopwords)

### 토큰화

In [84]:
# stopword 제거한 뉴스 타이틀 토큰화
tokened_X = token.texts_to_sequences(X)
print(tokened_X[0])

[7070, 183, 8001, 1598, 1488, 1507, 7070, 39, 6954, 433, 58782, 190, 28946, 6954, 2929, 183, 7070, 58782, 10337, 75, 8001, 2826, 35835, 58782, 877, 4471, 880, 6954, 14397, 22359, 2196, 11189, 18966, 1536, 880, 892, 4426, 919]


### 문장 길이 맞춤

In [85]:
# 길이가 맞게 앞쪽에 0 붙여줌
max = 612
X_pad = pad_sequences(tokened_X, max)
print(X_pad[:10])

[[    0     0     0 ...   892  4426   919]
 [    0     0     0 ... 51835 34172  7964]
 [    0     0     0 ...  1553   220   185]
 ...
 [    0     0     0 ...  6125  2458     4]
 [    0     0     0 ...   180    35  1415]
 [    0     0     0 ... 14107   714    80]]


## 예측 결과 확인

In [86]:
category[0]

'Art_Culture'

In [87]:
predict = model.predict(X_pad)
print(predict[0])
print(category[np.argmax(predict[0])])

[3.6235626e-05 2.5077385e-05 2.4975673e-06 4.3224773e-08 9.9993503e-01
 4.5633952e-10 2.3479709e-08 1.1011249e-06]
Novel_Poem


In [88]:
 predict_category = [category[np.argmax(p)] for p in predict]
 print(predict_category)

['Novel_Poem', 'Novel_Poem', 'Novel_Poem', 'Novel_Poem', 'Novel_Poem', 'Novel_Poem', 'Novel_Poem', 'Novel_Poem', 'Novel_Poem', 'Novel_Poem', 'Novel_Poem', 'Novel_Poem', 'Novel_Poem', 'Novel_Poem', 'Novel_Poem', 'Novel_Poem', 'Health_Hobby_Leisure', 'Novel_Poem', 'Novel_Poem', 'Novel_Poem', 'Novel_Poem', 'Novel_Poem', 'Novel_Poem', 'Novel_Poem', 'Novel_Poem', 'Novel_Poem', 'Home_Cook_Beauty', 'Novel_Poem', 'Novel_Poem', 'Novel_Poem', 'Economic_Management', 'Art_Culture', 'Art_Culture', 'Novel_Poem', 'Novel_Poem', 'Novel_Poem', 'Novel_Poem', 'Novel_Poem', 'Novel_Poem', 'Novel_Poem', 'Religion_Mysticism', 'Novel_Poem', 'Novel_Poem', 'Novel_Poem', 'Novel_Poem', 'Novel_Poem', 'Economic_Management', 'Novel_Poem', 'Economic_Management', 'Novel_Poem', 'Novel_Poem', 'Novel_Poem', 'Novel_Poem', 'Novel_Poem', 'Novel_Poem', 'Novel_Poem', 'Novel_Poem', 'Novel_Poem', 'Novel_Poem', 'Novel_Poem', 'Novel_Poem', 'Novel_Poem', 'Novel_Poem', 'Novel_Poem', 'Novel_Poem', 'Novel_Poem', 'Novel_Poem', 'Novel_P

In [89]:
df['predict'] = predict_category

In [90]:
df['OX'] = df.apply(lambda x: True if x['predict']==x['category'] else False, axis=1)

In [96]:
df[-20:]

,title,summary,category,predict,OX
412,새 삶을 주는 인공장기,"저자는 인공장기의 개발과 현황에 대해 살펴보고, 그 특성과 기능, 반응 등에 대해 ...",Science,Science,True
413,플랜트 배관 데이터북,최신 KS/JIS/ANSI 규격에 의한 플랜트 배관 필수 데이터북이다. 플랜트 산업...,Science,Economic_Management,False
414,흙집 제대로 짓기,"'흙건축 시리즈' 두번째 책. 흙건축 기술 하나하나를 진지하게 탐구하고, 체계적으로...",Science,Science,True
415,표면 특성 기기 분석,현대의 표면분석 기기의 올바른 선택과 효율적인 사용을 위해 측정 장치의 기본 원리를...,Science,Science,True
416,최신 개정 KS규격에 따른 금속재료 핸드북,"현장 실무 활용서 시리즈 3권. 철강 및 비철금속 재료 데이터, 최신 열처리 표시와...",Science,Science,True
417,창의 수학 콘서트,독자들 스스로 수학적 문제 해결 능력을 향상시켜 입체적 사고력을 키우고 창의성을 계...,Science,Science,True
418,이것은 수학입니까?,1+1=2라는 것을 수학적으로 어떻게 증명할 수 있을까? 왜 음수 곱하기 양수는 음...,Science,Science,True
419,공업계측,"'중화학공업기술교재' 시리즈. 사내 조업 기술 훈련 교재로, 우리나라에서도 유수한 ...",Science,Science,True
420,핵심 화학반응공학,"회분식반응기, 연속교반탱크반응기, 플러그흐름반응기, 충전층반응기, 유동층, 미분 반...",Science,Science,True
421,정신건강론 (유수현 외),"본서는 대학생을 대상으로 정신건강이 무엇이며, 인생의 주기에 따라 관심을 두어야 할...",Science,Novel_Poem,False


In [92]:
df.category.value_counts()

Novel_Poem              68
Economic_Management     65
Health_Hobby_Leisure    61
Religion_Mysticism      58
Art_Culture             53
Science                 52
Home_Cook_Beauty        45
Reference_Book          30
Name: category, dtype: int64

In [93]:
df[['category', 'OX']].groupby(['category']).mean()

,OX
category,
Art_Culture,0.792453
Economic_Management,0.846154
Health_Hobby_Leisure,0.737705
Home_Cook_Beauty,0.822222
Novel_Poem,0.882353
Reference_Book,1.000000
Religion_Mysticism,0.948276
Science,0.711538


In [94]:
print(df['OX'].value_counts() / len(df['OX']))

True     0.835648
False    0.164352
Name: OX, dtype: float64
